# Extração de Imagens de Cicatrizes de Incêndios

## Bibliotecas usadas:

In [ ]:
import geopandas as gpd
import pandas as pd
import math
import os
import numpy as np
from datetime import timedelta


import matplotlib.pyplot as plt
from IPython.display import Image
from skimage.morphology import (
    binary_closing,
    binary_erosion,
    binary_dilation,
    remove_small_objects,
    remove_small_holes,
    disk
)
from shapely.geometry import box
import folium
import warnings
warnings.filterwarnings("ignore")

import rasterio
from rasterio import Affine
from rasterio.crs import CRS
#import rasterio.transform
from rasterio.windows import from_bounds
from rasterio.warp import Resampling, reproject, transform

import pystac_client
pystac_client.__version__

### Definindo catálogo e API (STAC client e data.inpe)

Para isso utilizaremos o serviço Spatio Temporal Asset Catalog (STAC) por meio de um client na linguagem de programação Python.

O endereço do serviço STAC do BDC é https://data.inpe.br/bdc/stac/v1/. 

In [5]:
servico  = "https://data.inpe.br/bdc/stac/v1/"   
catalogo  = pystac_client.Client.open(servico)

## Extração Pontual de Imagens:

### Importando Arquivo de Focos de Incêndio, aplicando filtros e transformando:

In [6]:
df = pd.read_csv("focos_incêndios_cerrado/focos_qmd_inpe_2024-08-01_2024-10-31_08.347472.csv") # Lendo arquivo que possui dados dos focos de incêndio

# Aplique os filtros
filtro = (
    (df["Estado"].isin(["MARANHÃO","MARANHAO", "TOCANTINS"])) &
    (df["RiscoFogo"] >= 0.85) &
    (df["FRP"] >= 70)
)

# Selecione apenas as linhas que atendem aos filtros
df_filtrado = df[filtro]


In [7]:
# Exibir as primeiras linhas do resultado
df_filtrado.head()

,DataHora,Satelite,Pais,Estado,Municipio,Bioma,DiaSemChuva,Precipitacao,RiscoFogo,Latitude,Longitude,FRP
493,2024/08/01 03:55:41,GOES-16,Brasil,TOCANTINS,TOCANTINÓPOLIS,Cerrado,24,0.0,1.0,-6.1830,-47.6133,76.3
1775,2024/08/01 11:45:43,GOES-16,Brasil,MARANHÃO,MIRADOR,Cerrado,27,0.0,1.0,-6.6401,-44.7424,99.4
1777,2024/08/01 11:55:41,GOES-16,Brasil,TOCANTINS,TOCANTINÓPOLIS,Cerrado,24,0.0,1.0,-6.1830,-47.6133,75.0
1778,2024/08/01 11:55:43,GOES-16,Brasil,MARANHÃO,MIRADOR,Cerrado,27,0.0,1.0,-6.6401,-44.7424,84.9
1780,2024/08/01 12:15:47,GOES-16,Brasil,MARANHÃO,FORTALEZA DOS NOGUEIRAS,Cerrado,77,0.0,1.0,-7.0958,-45.9938,91.1


In [8]:
df_filtrado['DataHora'] = pd.to_datetime(df_filtrado['DataHora'], utc=True, errors='coerce') 
# Converte a coluna 'DataHora' para datetime(Mais facil manipular datas e realizar operações)

In [9]:
'''
gpd.GeoDataFrame(...)
Cria um GeoDataFrame (objeto do geopandas) a partir do DataFrame focos_filt.
Diferente de um pandas.DataFrame comum, ele entende geometrias (pontos, polígonos, linhas) e permite fazer operações espaciais (interseção, buffer, dissolver, reprojetar etc.).

geometry=gpd.points_from_xy(...)
Converte as colunas de Longitude e Latitude em uma coluna de geometria do tipo Point. Cada foco de incêndio vira um ponto no espaço.

crs="EPSG:4326"
Define o sistema de referência espacial (CRS) como WGS84 (EPSG:4326), que é latitude/longitude em graus decimais — o padrão usado pelo INPE, STAC, Sentinel, etc.
'''
gdf = gpd.GeoDataFrame(
    df_filtrado,
    geometry=gpd.points_from_xy(df_filtrado['Longitude'], df_filtrado['Latitude']),
    crs="EPSG:4326"
)


### Transformando focos em áreas de interesse (AOIs) para o STAC:

In [20]:
# reprojetar para métrico antes de fazer buffer
focos_m = gdf.to_crs(epsg=3857)  # projeta para Web Mercator (metros)
focos_m['date'] = focos_m['DataHora'].dt.date

bboxes = []
for dt, sub in focos_m.groupby('date'):
    # Buffer de 5 km (5000 m)
    area = sub.buffer(5000).unary_union  # dissolve
    # Converter de volta para lat/long
    area4326 = gpd.GeoSeries([area], crs=3857).to_crs(4326).iloc[0]
    # Gerar bounding boxes (multi‑polígonos separados)
    geoms = list(area4326.geoms) if hasattr(area4326, 'geoms') else [area4326]
    for geom in geoms:
        minx, miny, maxx, maxy = geom.bounds
        bboxes.append({
            'date': pd.to_datetime(dt),
            'bbox': [minx, miny, maxx, maxy],
            'n_focos': len(sub),
            'frp_median': sub['FRP'].median(),
            'risco_mean': sub['RiscoFogo'].mean(),
        })

### Elencando as áreas de interesse mais promissoras:

In [21]:
bboxes_df = pd.DataFrame(bboxes).copy()

In [22]:
bboxes_df.head(5)

,date,bbox,n_focos,frp_median,risco_mean
0,2024-08-01,"[-50.300065764205975, -11.991029282314292, -50...",292,94.55,0.985959
1,2024-08-01,"[-48.416115764205976, -11.69648659015319, -48....",292,94.55,0.985959
2,2024-08-01,"[-50.591135764205966, -11.081841564894722, -50...",292,94.55,0.985959
3,2024-08-01,"[-50.52584576420597, -10.898858859661933, -50....",292,94.55,0.985959
4,2024-08-01,"[-49.47671576420598, -10.447374280307615, -49....",292,94.55,0.985959


In [23]:

#### CÓDIDO DUPLICADO PARA TESTE


# garantir tipos, (tranformando as datas em datetime e os números em float, o que permite operações numéricas e comparações)
bboxes_df["date"] = pd.to_datetime(bboxes_df["date"])
for c in ["frp_median", "risco_mean", "n_focos"]:
    bboxes_df[c] = pd.to_numeric(bboxes_df[c], errors="coerce")
#_____________________________________________________________



# 2) (opcional) tratar NaN (se houver, o que não deve acontecer)
bboxes_df["frp_median"] = bboxes_df["frp_median"].fillna(0)
bboxes_df["risco_mean"] = bboxes_df["risco_mean"].fillna(0)
bboxes_df["n_focos"] = bboxes_df["n_focos"].fillna(0)
#_____________________________________________________________



# 3) (opcional) criar um score combinado para ordenar melhor
# normaliza colunas (min-max por TODO o período; se preferir, normalize por dia)
def minmax(x):
    x = x.astype(float)
    if x.max() == x.min():
        return np.zeros_like(x, dtype=float)
    return (x - x.min()) / (x.max() - x.min())

bboxes_df["frp_n"]   = minmax(bboxes_df["frp_median"])
bboxes_df["risco_n"] = minmax(bboxes_df["risco_mean"])
bboxes_df["nfocos_n"]= minmax(bboxes_df["n_focos"])
#_____________________________________________________________


# peso ajustável; aqui dou mais peso a FRP e n_focos
w_frp, w_risco, w_nfocos = 0.5, 0.2, 0.3
bboxes_df["score"] = (w_frp*bboxes_df["frp_n"] +
                      w_risco*bboxes_df["risco_n"] +
                      w_nfocos*bboxes_df["nfocos_n"])
#_____________________________________________________________



# 4) ordenar global (útil pra inspecionar)
bboxes_df = bboxes_df.sort_values(["date", "score"], ascending=[True, False]).reset_index(drop=True)
#_____________________________________________________________



# 5) RANQUEAR por dia (top-K por data)
TOPK = 1  # ajuste como quiser
topk_por_dia = (
    bboxes_df
    .groupby("date", group_keys=False)
    .apply(lambda df: df.nlargest(TOPK, "score"))
    .reset_index(drop=True)
)
#_____________________________________________________________


priorizadas = topk_por_dia


In [24]:
priorizadas.head(5)

,date,bbox,n_focos,frp_median,risco_mean,frp_n,risco_n,nfocos_n,score
0,2024-08-01,"[-50.300065764205975, -11.991029282314292, -50...",292,94.55,0.985959,0.340932,0.815854,0.083431,0.358666
1,2024-08-02,"[-48.28621576420598, -13.152941346694208, -48....",741,115.20,0.994035,0.750248,0.921772,0.215335,0.624079
2,2024-08-03,"[-50.680715764205985, -12.156412210162701, -50...",599,104.30,0.998364,0.534192,0.978543,0.173619,0.514891
3,2024-08-04,"[-50.65201576420597, -11.302448060057298, -50....",751,116.30,0.998056,0.772052,0.974504,0.218273,0.646408
4,2024-08-05,"[-49.72691576420597, -12.2094034584012, -49.63...",935,105.00,0.992278,0.548067,0.898729,0.272327,0.535477


### Consultando o STAC do INPE com as AOIs:

In [25]:

# =========================
# Parâmetros de teste
# =========================
SERVICO_STAC = "https://data.inpe.br/bdc/stac/v1/"
COLECAO = "S2_L2A-1"
cloud_lt = 15                 # mais restrito p/ teste
pre_days = 6
post_days = 6
DATE_WIN = 20           
MAX_CLUSTERS = 100           
MAX_ITEMS_PER_CLUSTER = 100  # limitar nº de itens analisados por cluster

catalogo = pystac_client.Client.open(SERVICO_STAC)
cloud_filter = {"eo:cloud_cover": {"lt": cloud_lt}}

# -------------------------
# Utils para "mesma região"
# -------------------------
def mgrs_or_bbox_key(item):
    """Tenta pegar o tile MGRS do item; se não existir,
    volta uma 'assinatura' via bbox arredondada."""
    props = item.properties or {}

    # chaves possíveis (varia entre catálogos)
    candidates = [
        "s2:tile_id", "s2:mgrs_tile",
        "mgrs:tile", "mgrs:code", "grid:code",
        # combinação UTM/latitude_band/grid_square às vezes aparece
        ("sentinel:utm_zone", "sentinel:latitude_band", "sentinel:grid_square"),
    ]

    for c in candidates:
        if isinstance(c, tuple):
            a, b, d = (props.get(c[0]), props.get(c[1]), props.get(c[2]))
            if a and b and d:
                return f"MGRS:{a}{b}{d}"
        else:
            if props.get(c):
                return f"MGRS:{props[c]}"

    # fallback: chave pelo bbox (arredondado)
    minx, miny, maxx, maxy = item.bbox
    r = lambda v: round(float(v), 4)
    return f"BBOX:{r(minx)},{r(miny)},{r(maxx)},{r(maxy)}"


# ---------- Utils de tempo (padrão: UTC tz-aware) ----------
def to_utc_ts(x):
    """
    Converte qualquer objeto de data (str/datetime/Timestamp) para pandas.Timestamp
    tz-aware em UTC. Retorna None se não der pra converter.
    """
    if x is None:
        return None
    ts = pd.to_datetime(x, utc=True, errors="coerce")
    if ts is None:
        return None
    # se por algum motivo vier sem tz (raro c/ utc=True), localiza em UTC
    if getattr(ts, "tz", None) is None:
        ts = ts.tz_localize("UTC")
    return ts

def item_datetime_utc(item):
    """
    Pega a melhor data/hora de um item STAC e devolve tz-aware em UTC.
    Prioriza 'datetime'; se não houver, tenta 'start_datetime'/'end_datetime'
    e, por último, item.datetime.
    """
    p = getattr(item, "properties", {}) or {}
    if p.get("datetime"):
        return to_utc_ts(p["datetime"])
    # alguns catálogos usam intervalo:
    if p.get("start_datetime"):
        return to_utc_ts(p["start_datetime"])
    if p.get("end_datetime"):
        return to_utc_ts(p["end_datetime"])
    # fallback
    if getattr(item, "datetime", None) is not None:
        return to_utc_ts(item.datetime)
    return None

# substitui a sua antiga `to_date`
def to_date(item):
    return item_datetime_utc(item)

# -------------------------
# Busca + pareamento
# -------------------------
pairs = []  # lista final com pares antes/depois

print(f"Buscando pares p/ até {MAX_CLUSTERS} clusters...")

for _, row in priorizadas.head(MAX_CLUSTERS).iterrows():
    print(f"Cluster {row['date'].date()} (score={row['score']:.3f})...")
    # row["date"] possivelmente vem como date (sem hora/tz). Coloque em UTC:
    foco_date = to_utc_ts(row["date"])              # vira 00:00:00Z daquele dia
    # caso queira centralizar no meio do dia, você pode somar horas:
    # foco_date = foco_date + pd.Timedelta(hours=12)

    # janelinha curta p/ teste (aqui pode ser DATE_WIN tz-naive,
    # pois vamos usar só para compor strings de busca do STAC)
    date_range = f"{(foco_date.tz_convert('UTC') - timedelta(days=DATE_WIN)).date()}/" \
                 f"{(foco_date.tz_convert('UTC') + timedelta(days=DATE_WIN)).date()}"

    search = catalogo.search(
        bbox=row["bbox"],
        collections=[COLECAO],
        query=cloud_filter,
        datetime=date_range
    )
    items = list(search.get_items())
    print(f"  → {len(items)} itens encontrados.")
    if not items:
        print("⚠️ Nenhum item encontrado.")
        continue

    items = items[:MAX_ITEMS_PER_CLUSTER]
    
    ############################################################### Aqui tudo funcionando #######################################################

    # agrupar por região
    region_groups = {}
    for it in items:
        key = mgrs_or_bbox_key(it)
        region_groups.setdefault(key, []).append(it)

    # cortes tz-aware
    pre_cut  = foco_date - timedelta(days=pre_days)
    post_cut = foco_date + timedelta(days=post_days)

    for region_key, its in region_groups.items():
        # ordenar por datetime em UTC
        its_sorted = sorted(its, key=to_date)

        # candidatos pré/pós (tudo tz-aware)
        pre_candidates  = [it for it in its_sorted if to_date(it) is not None and to_date(it) <= pre_cut]
        post_candidates = [it for it in its_sorted if to_date(it) is not None and to_date(it) >= post_cut]

        pre_item  = pre_candidates[-1] if pre_candidates else None
        post_item = post_candidates[0]  if post_candidates else None

        if pre_item and post_item:
            pairs.append({
                    "cluster_date": foco_date.tz_convert("UTC").date(),  # ✅ nome certo e date()
                    "region_key":   region_key,
                    "bbox":         row["bbox"],
                    "frp_median":   row.get("frp_median"),
                    "risco_mean":   row.get("risco_mean"),
                    "pre_id":       pre_item.id,
                    "pre_datetime": to_date(pre_item),   # tz-aware
                    "pre_cloud":    pre_item.properties.get("eo:cloud_cover"),
                    "post_id":      post_item.id,
                    "post_datetime":to_date(post_item),  # tz-aware
                    "post_cloud":   post_item.properties.get("eo:cloud_cover"),
                })

Buscando pares p/ até 100 clusters...
Cluster 2024-08-01 (score=0.359)...
  → 9 itens encontrados.
Cluster 2024-08-02 (score=0.624)...


KeyboardInterrupt: 

In [ ]:
pairs_df = pd.DataFrame(pairs)
pairs_df = pairs_df.sort_values(["cluster_date","region_key"])


#### Pegando imagens não repetidas:

In [21]:
pairs_df = pairs_df.drop_duplicates(subset=["pre_id", "post_id"]) # remover duplicatas exatas de imagens(as vezes existem multiplos alertas de incêndio para o mesmo foco)


#### Tentando vizualizar pares de imagens encontrados:

In [23]:
import io, requests
from PIL import Image
import matplotlib.pyplot as plt
import pystac_client
import pandas as pd

SERVICO_STAC = "https://data.inpe.br/bdc/stac/v1/"
COLECAO = "S2_L2A-1"

catalogo = pystac_client.Client.open(SERVICO_STAC)

def pick_quicklook_url(item):
    # tenta encontrar uma pré-visualização pronta
    for key in ["thumbnail","visual","overview","quicklook"]:
        if key in item.assets:
            return item.assets[key].href
    return None  # sem quicklook disponível

def fetch_image(url):
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return Image.open(io.BytesIO(r.content))

def show_pair(idx):
    row = pairs_df.iloc[idx]

    def fetch_item(item_id):
        search = catalogo.search(
            collections=[COLECAO],
            ids=[item_id]
        )
        itens = list(search.get_items())
        return itens[0] if itens else None

    pre_item  = fetch_item(row["pre_id"])
    post_item = fetch_item(row["post_id"])

    pre_url  = pick_quicklook_url(pre_item)
    post_url = pick_quicklook_url(post_item)

    # mostra o índice real do DataFrame junto do índice passado
    print(f"[{idx}] (pairs_df index={row.name}) cluster_date={row['cluster_date']}, region={row['region_key']}")
    print("pre:", row["pre_id"], row["pre_datetime"], "cloud:", row["pre_cloud"], "→", pre_url)
    print("post:", row["post_id"], row["post_datetime"], "cloud:", row["post_cloud"], "→", post_url)

    if pre_url and post_url:
        im_pre  = fetch_image(pre_url)
        im_post = fetch_image(post_url)

        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1); plt.title(f"PRE (row {row.name})");  plt.axis("off"); plt.imshow(im_pre)
        plt.subplot(1,2,2); plt.title(f"POST (row {row.name})"); plt.axis("off"); plt.imshow(im_post)
        plt.tight_layout()
        plt.show()
    else:
        print("Quicklook não disponível para algum dos itens; tente o fluxo RGB (B04,B03,B02).")




In [26]:
for i in range(0, 3):
    show_pair(i)

NameError: name 'show_pair' is not defined

## Gerando Df_final com relações das imagens usadas

In [ ]:


# exemplo:
lista1 = [n_1,n_2,n_3]  # substitua n_1, n_2, n_3 pelos índices desejados com nas imagens de interesse acima (Esses itens correpondem aos itens visualizados com show_pair)

# seleciona as linhas desejadas de cada dataset
subset1 = pairs_df.iloc[lista1]

# concatena preservando as colunas
df_final_images = pd.concat([subset1], ignore_index=True)

import os
if not os.path.exists("Imagens_e_indices"):
    os.makedirs("Imagens_e_indices")

df_final_images.to_csv("Imagens_e_indices/coordenadas_e_informacoes_cenas_usadas.csv", index=False)
